In [1]:
import os
import re
import ast
import json
import spacy
import pickle
import difflib
import requests
import datetime
import jsonlines
import numpy as np
import pandas as pd
from utils.shared_configs import LLAMA_MODEL_PATH, ZEHPYR_MODEL_PATH, get_sampling_params, initialize_llm
nlp = spacy.load("en_core_web_sm")

2025-05-02 15:36:21.232213: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 15:36:21.604006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746200181.660856  230775 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746200181.687032  230775 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-02 15:36:22.261769: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
llm = initialize_llm(model_path=LLAMA_MODEL_PATH, tokenizer_path=LLAMA_MODEL_PATH)
sampling_params = get_sampling_params(max_tokens=200, temperature=0.6, top_p=0.9, stops=["</s>", "\n}"])

INFO 05-02 15:36:42 __init__.py:207] Automatically detected platform cuda.
INFO 05-02 15:37:03 config.py:549] This model supports multiple tasks: {'classify', 'embed', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
WARNING 05-02 15:37:03 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 05-02 15:37:03 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 05-02 15:37:03 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/llama3/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f', speculative_config=None, tokenizer='/datasets/ai/llama3/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f', skip

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 05-02 15:37:42 model_runner.py:1115] Loading model weights took 14.9888 GB
INFO 05-02 15:37:44 worker.py:267] Memory profiling takes 1.60 seconds
INFO 05-02 15:37:44 worker.py:267] the current vLLM instance can use total_gpu_memory (44.40GiB) x gpu_memory_utilization (0.90) = 39.96GiB
INFO 05-02 15:37:44 worker.py:267] model weights take 14.99GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 23.70GiB.
INFO 05-02 15:37:44 executor_base.py:111] # cuda blocks: 12136, # CPU blocks: 2048
INFO 05-02 15:37:44 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 1.48x
INFO 05-02 15:37:46 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:18<00:00,  1.93it/s]

INFO 05-02 15:38:04 model_runner.py:1562] Graph capturing finished in 18 secs, took 0.26 GiB
INFO 05-02 15:38:04 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 21.52 seconds


### Load WN-Salience

In [2]:
# wn_salience_train = "/project/pi_wenlongzhao_umass_edu/8/data/WN-Salience-2/SED_train.json"
# with open(wn_salience_train) as f:
#     sed_outputs = json.load(f)

In [20]:
train_path = "/work/pi_wenlongzhao_umass_edu/8/riya/TrainValDataSplit/WN_salience_TrainSplit.pkl"
val_path = "/work/pi_wenlongzhao_umass_edu/8/riya/TrainValDataSplit/WN_salience_ValSplit.pkl"

In [21]:
with open(val_path, "rb") as f:
    wn_val = pickle.load(f)

/tmp/ipykernel_848945/2559814351.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  wn_val = pickle.load(f)


In [22]:
wn_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15644 entries, 0 to 15643
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   text             15644 non-null  object        
 1   date             15644 non-null  datetime64[ns]
 2   title            15644 non-null  object        
 3   entity title     15603 non-null  object        
 4   entity salience  15644 non-null  int64         
 5   offsets          15644 non-null  object        
 6   url              15544 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 855.7+ KB


In [23]:
wn_val.head()

,text,date,title,entity title,entity salience,offsets,url
0,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Australian,1,"(50, 60)",/wiki/Australia
1,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Gliders,1,"(61, 68)",/wiki/Australia_women%27s_national_wheelchair_...
2,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Chinese,1,"(78, 85)",/wiki/China
3,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,wheelchair basketball,1,"(103, 124)",/wiki/Wheelchair_basketball
4,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Sydney,1,"(221, 227)",/wiki/Sydney


## Pointwise EL

### Preprocessing
1. Rename columns
2. Add surrounding context
3. Assign WikiID based on Wiki url

In [24]:
column_mapping = {
    'text': 'article_text',
    'title': 'article_title',
    'entity title': 'entity_title',
    'entity salience': 'entity_salience'
}
wn_val.rename(columns=column_mapping, inplace=True)

In [25]:
wn_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15644 entries, 0 to 15643
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   article_text     15644 non-null  object        
 1   date             15644 non-null  datetime64[ns]
 2   article_title    15644 non-null  object        
 3   entity_title     15603 non-null  object        
 4   entity_salience  15644 non-null  int64         
 5   offsets          15644 non-null  object        
 6   url              15544 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 855.7+ KB


In [26]:
wn_val = wn_val.dropna(subset=['entity_title'])
wn_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15603 entries, 0 to 15643
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   article_text     15603 non-null  object        
 1   date             15603 non-null  datetime64[ns]
 2   article_title    15603 non-null  object        
 3   entity_title     15603 non-null  object        
 4   entity_salience  15603 non-null  int64         
 5   offsets          15603 non-null  object        
 6   url              15543 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 975.2+ KB


In [27]:
wn_val_subset = wn_val.head(250).copy()

In [28]:
wn_val_subset.head()

,article_text,date,article_title,entity_title,entity_salience,offsets,url
0,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Australian,1,"(50, 60)",/wiki/Australia
1,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Gliders,1,"(61, 68)",/wiki/Australia_women%27s_national_wheelchair_...
2,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Chinese,1,"(78, 85)",/wiki/China
3,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,wheelchair basketball,1,"(103, 124)",/wiki/Wheelchair_basketball
4,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Sydney,1,"(221, 227)",/wiki/Sydney


In [29]:
def spacy_sent_tokenize(text):
    return [sent.text for sent in nlp(text).sents]

def extract_surrounding_context(article_text, offset, entity_title, n=1):
    """
    Return n sentences before and after the entity sentence, 
    with entity mention highlighted with ###mention###.
    """
    start, end = offset
    sents = list(nlp(article_text).sents)

    for i, sent in enumerate(sents):
        if sent.start_char <= start < sent.end_char:
            # Pick n sentences before and after
            begin = max(i - n, 0)
            end_ = min(i + n + 1, len(sents))
            context_sentences = [s.text for s in sents[begin:end_]]
            context = " ".join(context_sentences)
            entity_pattern = re.escape(entity_title)
            highlighted_context = re.sub(
                entity_pattern,
                f"###{entity_title}###",
                context,
                count=1  # Only highlight the first occurrence
            )
            return highlighted_context
    return ""

In [30]:
wn_val_subset['surrounding_context'] = wn_val_subset.apply(lambda row: extract_surrounding_context(row['article_text'], row['offsets'], row['entity_title'], n=2), axis=1)

In [31]:
wn_val_subset.head()

,article_text,date,article_title,entity_title,entity_salience,offsets,url,surrounding_context
0,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Australian,1,"(50, 60)",/wiki/Australia,"Homebush Bay, New South Wales —Earlier today, ..."
1,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Gliders,1,"(61, 68)",/wiki/Australia_women%27s_national_wheelchair_...,"Homebush Bay, New South Wales —Earlier today, ..."
2,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Chinese,1,"(78, 85)",/wiki/China,"Homebush Bay, New South Wales —Earlier today, ..."
3,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,wheelchair basketball,1,"(103, 124)",/wiki/Wheelchair_basketball,"Homebush Bay, New South Wales —Earlier today, ..."
4,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Sydney,1,"(221, 227)",/wiki/Sydney,"Homebush Bay, New South Wales —Earlier today, ..."


In [32]:
def get_wiki_id(url):
    if not url:
        return None
    
    # title_match = re.search(r"wiki/(.+)$", url)
    title_match = re.search(r"wiki/([^#?]+)", url)
    if not title_match:
        return None
    
    title = title_match.group(1)
    api_url = f"https://en.wikipedia.org/w/api.php?action=query&titles={title}&format=json"
    
    try:
        response = requests.get(api_url)
        response_data = response.json()
        pages = response_data.get("query", {}).get("pages", {})
        
        if pages:
            return next(iter(pages))  # Extract the first (and only) page ID
    except Exception as e:
        print(f"Error fetching Wiki ID for {url}: {e}")
    
    return None

wn_val_subset['gt_wiki_id'] = wn_val_subset['url'].apply(get_wiki_id)

Error fetching Wiki ID for https://en.wikipedia.org/wiki/President_of_the_United_States#Eligibility: Expecting value: line 1 column 1 (char 0)


In [35]:
wn_val_subset.head()

,article_text,date,article_title,entity_title,entity_salience,offsets,url,surrounding_context,gt_wiki_id
0,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Australian,1,"(50, 60)",/wiki/Australia,"Homebush Bay, New South Wales —Earlier today, ...",4689264
1,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Gliders,1,"(61, 68)",/wiki/Australia_women%27s_national_wheelchair_...,"Homebush Bay, New South Wales —Earlier today, ...",32339793
2,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Chinese,1,"(78, 85)",/wiki/China,"Homebush Bay, New South Wales —Earlier today, ...",5405
3,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,wheelchair basketball,1,"(103, 124)",/wiki/Wheelchair_basketball,"Homebush Bay, New South Wales —Earlier today, ...",848348
4,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Sydney,1,"(221, 227)",/wiki/Sydney,"Homebush Bay, New South Wales —Earlier today, ...",27862


In [36]:
wn_val_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 0 to 255
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   article_text         250 non-null    object        
 1   date                 250 non-null    datetime64[ns]
 2   article_title        250 non-null    object        
 3   entity_title         250 non-null    object        
 4   entity_salience      250 non-null    int64         
 5   offsets              250 non-null    object        
 6   url                  241 non-null    object        
 7   surrounding_context  250 non-null    object        
 8   gt_wiki_id           239 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 19.5+ KB


In [34]:
wn_val_subset.to_csv('wn_val_subset_5_1.csv', index=False)

In [7]:
wn_val_subset = pd.read_csv('wn_val_subset_5_1.csv', dtype={'gt_wiki_id': 'Int64'})
wn_val_subset.head()

,article_text,date,article_title,entity_title,entity_salience,offsets,url,surrounding_context,gt_wiki_id
0,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Australian,1,"(50, 60)",/wiki/Australia,"Homebush Bay, New South Wales —Earlier today, ...",4689264
1,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Gliders,1,"(61, 68)",/wiki/Australia_women%27s_national_wheelchair_...,"Homebush Bay, New South Wales —Earlier today, ...",32339793
2,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Chinese,1,"(78, 85)",/wiki/China,"Homebush Bay, New South Wales —Earlier today, ...",5405
3,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,wheelchair basketball,1,"(103, 124)",/wiki/Wheelchair_basketball,"Homebush Bay, New South Wales —Earlier today, ...",848348
4,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Sydney,1,"(221, 227)",/wiki/Sydney,"Homebush Bay, New South Wales —Earlier today, ...",27862


### Candidate retrieval

In [3]:
with open('/work/pi_wenlongzhao_umass_edu/8/aranade/696-detecting-salient-entities/prep_kb/filtered_kb_4_22.json') as f:
    filtered_kb = json.load(f)
    
alias_kb = {}
for entry in filtered_kb:
    key = entry['mentions'].lower()
    alias_kb.setdefault(key, []).append(entry)
    # cid = entry['wiki_id']
    # candidate_freq[cid] = candidate_freq.get(cid, 0) + 1

# max_freq, min_freq = max(candidate_freq.values()), min(candidate_freq.values())

len(alias_kb)

29037

In [4]:
def retrieve_candidates(mention, alias_kb):
    return alias_kb.get(mention.lower())

In [7]:
# import copy

# def retrieve_candidates(mention, cutoff=0.8, alpha=0.5, k=10):
#     mention_lower = mention.lower()
#     close_keys = difflib.get_close_matches(mention_lower, alias_kb.keys(), n=len(alias_kb), cutoff=cutoff)

#     # de‑dup while collecting
#     seen, uniq = set(), []
#     for key in close_keys:
#         for cand in alias_kb[key]:
#             cid = cand['cand_wiki_id']
#             # already collected
#             if cid in seen:
#                 continue
#             seen.add(cid)

#             new_cand = copy.deepcopy(cand)
#             # weighted score = α·fuzzy + (1‑α)·norm_freq
#             fuzzy = difflib.SequenceMatcher(None, mention_lower, cand['mention'].lower()).ratio()
#             freq  = candidate_freq[cid]
#             norm_freq = (freq - min_freq) / (max_freq - min_freq) if max_freq > min_freq else 1
#             new_cand['weighted_score'] = alpha * fuzzy + (1 - alpha) * norm_freq
#             uniq.append(new_cand)

#     return sorted(uniq, key=lambda c: c['weighted_score'], reverse=True)[:k]

### PROMPT TEMPLATE  (point‑wise)

In [11]:
def pointwise_el_prompt(entity, candidate):
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for: 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. <|eot_id|><|start_header_id|>user<|end_header_id|>\n Now, I will provide you with a mention, its context, and a candidate entity. The mention in the context is highlighted with '###'. mention: {entity['entity_title']} context: {entity['surrounding_context']} candidate entity: {candidate['title']}.{candidate['text'][:250]} Output the JSON object in a single line without any line breaks, indentation, or extra spaces. Only output the JSON object. Do not output anything else. Here is the JSON structure to follow: {{"final_decision": "yes or no", "reasoning": "your explanation here"}} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""" 
    return prompt

In [54]:
# def pointwise_el_chat(entity, candidate):
#     return [
#         {
#             "role": "user", 
#             "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Now, I will provide you with a mention, its context, and a candidate entity. The mention in the context is highlighted with '###'. mention: {entity['entity_title']} context: {entity['surrounding_context']}" candidate entity: {candidate['title']}. {candidate['text'][:300]}. Remember, the final answer must be \"yes\" when you believe the available information is insufficient or uncertain. Please analyze the information and determine if the mention and the candidate entity are related. Output the JSON object in a single line without any line breaks, indentation, or extra spaces. Only output the JSON object. Do not output anything else. Here is the JSON structure to follow:"{{"final_decision": "yes or no","reasoning": "your explanation here"}}""")
#         }
#     ]

In [ ]:
def pointwise_el_chat(entity, candidate):
    return [
        {
            "role": "user", 
            "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Now, I will provide you with a mention, its context, and a candidate entity. The mention in the context is highlighted with '###'. mention: {entity['entity_title']} context: {entity['surrounding_context']}" candidate entity: {candidate['title']}. {candidate['text'][:300]}. Remember, the final answer must be \"yes\" when you believe the available information is insufficient or uncertain. Please analyze the information and determine if the mention and the candidate entity are related. Output the JSON object in a single line without any line breaks, indentation, or extra spaces. Only output the JSON object. Do not output anything else. Here is the JSON structure to follow:"{{"final_decision": "yes or no","reasoning": "your explanation here"}}""")
        }
    ]

In [6]:
def pointwise_el_chat(entity, candidate):
    return [
        {
            "role": "user", 
            "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Your job is to decide whether a candidate should stay on the shortlist for an entity in context. 1. Output **"yes"** if the candidate is clearly the correct entity or you are uncertain / information is insufficient. Output **"no"** only if the candidate is clearly impossible. Now, I will provide you with an entity, its context, and a candidate with its description. The entity in the context is highlighted with '###'. entity: {entity['entity_title']} context: {entity['surrounding_context']}" candidate: {candidate['title']}. {candidate['text'][:300]}. Remember, the final answer must be \"yes\" when you believe the available information is insufficient or uncertain. Please analyze the information and determine if the entity and the candidate are related. Keep the answer to exactly one compact JSON object on a single line, no extra spaces: {{"final_decision":"yes|no","reasoning":""}}""")
        }
    ]

### Sample run

In [8]:
et = wn_val_subset.iloc[0]
et['entity_title']

'Australian'

In [9]:
et['article_text']

"Homebush Bay, New South Wales —Earlier today, the Australian Gliders beat the Chinese women's national wheelchair basketball team 57–45 at the Rollers & Gliders World Challenge taking place at the Sport Centre at the Sydney Olympic Park.China won the game's opening tip off but did not score in their first time down the floor. Australia's Sarah Stewart took and missed Australia's first shot. China's 1.5 point player Li Yanhua eventually made the game's first basket. Cobi Crispin made Australia's first basket at 6:55 left in the first quarter, successfully turning her basket into a three point play by making her free throw as a result of a personal foul. Aggressive first quarter play resulted in four Chinese personal fouls with 5:21 left. The first quarter ended 13–9 in China's favour.The first half ended 24–20, with China leading. China's lead came with a high level of energy, the bench loudly supporting their players, few substitutions, lots of on the court talk, playing a half court 

In [10]:
candidates=retrieve_candidates(et['entity_title'], alias_kb)
len(candidates)

58

In [13]:
prompt1=pointwise_el_chat(et, candidates[0])
op1 = llm.chat(messages=prompt1, sampling_params=sampling_params)
op1[0].outputs[0].text

INFO 05-02 17:33:50 chat_utils.py:332] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.34s/it, est. speed input: 375.09 toks/s, output: 38.18 toks/s]


'{"final_decision":"yes","reasoning":"Contextually, the entity ###Australian### Gliders refers to the Australian women\'s national wheelchair basketball team, which is directly related to the country of Australia, supporting the candidate\'s connection to the entity."}'

In [20]:
decision(op1[0].outputs[0].text)

True

In [17]:
prompt2=pointwise_el_prompt(et, candidates[0])
op2 = llm.generate(prompts=prompt2, sampling_params=sampling_params)
op2[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it, est. speed input: 293.55 toks/s, output: 43.70 toks/s]


'\n\n{"final_decision": "yes", "reasoning": "The mention ###Australian### refers to the team, not the country, as it is in the context of a specific sports event involving a national team, and the context is a match between Australia\'s team and China\'s team."}'

In [21]:
decision(op2[0].outputs[0].text)

True

In [19]:
# def decision(text: str) -> bool:
#     try:
#         text = text.strip()

#         # If text already starts with { and ends with }, treat it directly
#         if text.startswith("{") and text.endswith("}"):
#             data = json.loads(text)
#             return data.get("final_decision", "").strip().lower() == "yes"
        
#         # fallback: try to find first {...} block manually
#         first_brace = text.find("{")
#         last_brace = text.rfind("}")
#         if first_brace != -1 and last_brace != -1:
#             json_chunk = text[first_brace:last_brace+1]
#             data = json.loads(json_chunk)
#             return data.get("final_decision", "").strip().lower() == "yes"

#         raise ValueError("No JSON block found")

#     except Exception as e:
#         print(f"[decision error] Could not parse decision from: {text[:100]} — {e}")
#         return False

def decision(text: str) -> bool:
    """
    Extract the LLM’s yes/no verdict.
    If extraction fails, default to True (‘yes’) to maintain high recall.
    """
    try:
        if not text:
            raise ValueError("empty text")

        # Strip ``` or ```json fences and surrounding whitespace
        text = re.sub(r"^```(?:json)?|```$", "", text.strip(), flags=re.MULTILINE).strip()

        # ---------- 1. strict‑ish JSON ----------
        if text.startswith("{") and text.endswith("}"):
            try:
                data = json.loads(text)
            except json.JSONDecodeError:
                # fix single quotes & trailing commas, then retry
                cleaned = re.sub(r"(')", '"', text)
                cleaned = re.sub(r",\s*([}\]])", r"\1", cleaned)
                data = json.loads(cleaned)

            for k in ("final_decision", "finalDecision", "decision"):
                if k in data:
                    return str(data[k]).strip().lower() == "yes"

        # ---------- 2. regex fallback ----------
        m = re.search(
            r'''(?:final[_\s]?decision|finalDecision|decision)  # key variants
                \s*["']?\s*:\s*["']?\s*
                (?P<ans>yes|no)                                # value
            ''',
            text,
            flags=re.IGNORECASE | re.VERBOSE,
        )
        if m:
            return m.group("ans").lower() == "yes"

        # If we reach here, we didn’t find a usable answer
        raise ValueError("final_decision key not found")

    except Exception as e:
        print(f"[decision warning] Falling back to YES ‑ could not parse: {text[:120]} — {e}")
        return True

### Old implementation

In [ ]:
# # [(article_idx, ent_idx, cand_idx, prompt_str)]
# prompt_records = []          

# for art_idx, output in enumerate(sed_outputs):
#     for ent_idx, ent in enumerate(output["entities"]):
#         ent["candidates"] = retrieve_candidates(ent["entity_title"])
#         for cand_idx, cand in enumerate(ent["candidates"]):
#             prompt_records.append((art_idx, ent_idx, cand_idx, point_wise_el_prompt(ent, cand)))

In [ ]:
# len(prompt_records)

147315

In [ ]:
# with open(f"prompts/prompt_records_4_18_v3.json", "w", encoding="utf-8") as f:
#         json.dump(prompt_records, f, indent=2)
# print('Saved')

Saved


In [ ]:
# for chunk_start in range(0, len(prompt_records), BATCH):
#     chunk = prompt_records[chunk_start: chunk_start+BATCH]
#     prompts = [p[-1] for p in chunk]
#     responses = llm.generate(prompts=prompts, sampling_params=sampling_params)

#     timestamp = datetime.datetime.now().isoformat()
#     batch_out = []

#     for rec, resp in zip(chunk, responses):
#         art_idx, ent_idx, cand_idx, _ = rec
#         txt = resp.outputs[0].text
#         keep = decision(txt)

#         sed_outputs[art_idx]["entities"][ent_idx]["candidates"][cand_idx]["llm_decision"] = txt
#         sed_outputs[art_idx]["entities"][ent_idx]["candidates"][cand_idx]["relevant"] = keep

#         batch_out.append({
#             "article_idx": art_idx,
#             "entity_idx":  ent_idx,
#             "cand_idx":    cand_idx,
#             "keep":        keep,
#             "llm_text":    txt,
#             "timestamp":   timestamp
#         })

#     with open(f"outputs/pointwise/pointwise_filtered/batch_{chunk_start//BATCH+1}_{timestamp.replace(':','-')}.json", "w", encoding="utf-8") as f:
#         json.dump(batch_out, f, indent=2)
#     print(f'Saved batch {chunk_start//BATCH+1}')

In [ ]:
# with open(f"outputs/pointwise/pointwise_filtered/pointwise_sed_outputs_pre_pruning.json", "w", encoding="utf-8") as f:
#     json.dump(sed_outputs, f, indent=2)

In [17]:
for output in sed_outputs:
    for ent in output["entities"]:
        ent["candidates"] = [c for c in ent["candidates"] if c.get("relevant")]

In [19]:
with open(f"outputs/pointwise/pointwise_filtered/pointwise_sed_outputs.json", "w", encoding="utf-8") as f:
    json.dump(sed_outputs, f, indent=2)

In [21]:
chunk_7500 = prompt_records[chunk_start: chunk_start+BATCH]
prompts_7500 = [p[-1] for p in chunk]

In [24]:
chunk_7500[-1], prompts_7500[-1]

((272,
  6,
  4,
  '\n        <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n        You are an entity disambiguator. I\'ll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n\n        1. Context: Look at the surrounding text to understand the topic.\n\n        2. Categories: Consider the type of the entity (person, organization, location, etc.).\n\n        3. Modifiers: Pay attention to words or phrases that add details to the mention.\n\n        4. Co-references: Check other mentions of the same entity in the text.\n\n        5. Temporal and Geographical Factors: Consider when and where the text was written.\n\n        6. External Knowledge: Use knowledge from outside the text.\n\n        Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment.\n \n        <|eot_id|><|start_header_id|>user<|end_header_id|>\n\n        Now, I

### Dataframe based inference

In [22]:
skipped_rows = []
wn_val_subset['candidates'] = None
wn_val_subset['pre_pt_len_candidates'] = 0

# [(row_idx, cand_idx, prompt_str)]
for idx, row in wn_val_subset.iterrows():
    row_candidates = retrieve_candidates(row['entity_title'], alias_kb)
    if not row_candidates:
        skipped_rows.append((idx, row['entity_title']))
        continue
        
    wn_val_subset.at[idx, 'candidates'] = row_candidates
    wn_val_subset.at[idx, 'pre_pt_len_candidates'] = len(row_candidates)

In [23]:
wn_val_subset.to_csv('wn_val_subset_5_2_pre_pt.csv', index=False)

In [101]:
# wn_val_subset = pd.read_csv('wn_val_subset_5_2_pre_pt.csv', dtype={'gt_wiki_id': 'Int64'})

In [102]:
# wn_val_subset['candidates'] = wn_val_subset['candidates'].apply(
#     lambda x: ast.literal_eval(x) if isinstance(x, str) else x
# )

In [27]:
prompt_records = []

for idx, row in wn_val_subset.iterrows():
    row_candidates = row['candidates']
    if not isinstance(row_candidates, list) or not row_candidates:
        continue
        
    for cand_idx, cand in enumerate(row_candidates):
        prompt = pointwise_el_chat(row, cand)
        prompt_records.append({
            "row_idx": idx,
            "cand_idx": cand_idx,
            "messages": prompt
        })

In [28]:
len(prompt_records)

13583

In [29]:
prompt_records[0]

{'row_idx': 0,
 'cand_idx': 0,
 'messages': [{'role': 'user',
   'content': 'You are an entity disambiguator. I\'ll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Your job is to decide whether a candidate should stay on the shortlist for an entity in context. 1. Output **"yes"** if the candidate is clearly the correct entity or you are unce

In [30]:
prompt_df = pd.DataFrame(prompt_records)

In [31]:
prompt_df.head()

,row_idx,cand_idx,messages
0,0,0,"[{'role': 'user', 'content': 'You are an entit..."
1,0,1,"[{'role': 'user', 'content': 'You are an entit..."
2,0,2,"[{'role': 'user', 'content': 'You are an entit..."
3,0,3,"[{'role': 'user', 'content': 'You are an entit..."
4,0,4,"[{'role': 'user', 'content': 'You are an entit..."


In [32]:
BATCH = 7500
all_outputs = []

for chunk_start in range(0, len(prompt_df), BATCH):
    chunk = prompt_df.iloc[chunk_start:chunk_start + BATCH]
    messages_batch = list(chunk['messages'])
    # responses = llm.generate(prompts=messages_batch, sampling_params=sampling_params)
    responses = llm.chat(messages=messages_batch, sampling_params=sampling_params)

    outputs = []
    for (i, resp) in enumerate(responses):
        row = chunk.iloc[i]
        txt = resp.outputs[0].text
        keep = decision(txt)

        outputs.append({
            "row_idx": row['row_idx'],
            "cand_idx": row['cand_idx'],
            "keep": keep,
            "llm_text": txt
        })
    all_outputs.extend(outputs)

    print(f"Saved batch {chunk_start//BATCH + 1}")

Processed prompts: 100%|██████████| 7500/7500 [07:39<00:00, 16.33it/s, est. speed input: 8986.40 toks/s, output: 1228.56 toks/s]


[decision warning] Falling back to YES ‑ could not parse: {"final_decision":"no","reasoning":"The context mentions the 'left-wing Green Party', which suggests a political party w — Expecting ',' delimiter: line 1 column 63 (char 62)
Saved batch 1


Processed prompts: 100%|██████████| 6083/6083 [07:02<00:00, 14.41it/s, est. speed input: 8896.66 toks/s, output: 1193.77 toks/s]


Saved batch 2


In [33]:
import copy

wn_val_subset['candidates_after_pointwise'] = wn_val_subset['candidates'].apply(
    lambda x: copy.deepcopy(x) if isinstance(x, list) else []
)

In [34]:
output_df = pd.DataFrame(all_outputs)

for _, row in output_df.iterrows():
    row_idx = row['row_idx']
    cand_idx = row['cand_idx']
    
    wn_val_subset.at[row_idx, 'candidates_after_pointwise'][cand_idx]['llm_decision'] = row['llm_text']
    wn_val_subset.at[row_idx, 'candidates_after_pointwise'][cand_idx]['relevant'] = row['keep']

In [35]:
wn_val_subset.to_csv('wn_val_subset_5_2_pointwise_pre_pruning.csv', index=False)

In [78]:
# pre_pruning_df = pd.read_csv('wn_val_subset_5_1_pointwise_pre_pruning.csv')

In [36]:
wn_val_subset.iloc[0].candidates_after_pointwise

[{'mentions': 'australian',
  'wiki_id': 4689264,
  'title': 'Australia',
  'text': "Australia, officially the Commonwealth of Australia, is a sovereign country comprising the mainland of the Australian continent, the island of Tasmania, and numerous smaller islands. It is the largest country in Oceania and the world's sixth-largest country by total area. The neighbouring countries are Papua New Guinea, Indonesia, and East Timor to the north; the Solomon Islands and Vanuatu to the north-east; and New Zealand to the south-east. The population of million is highly urbanised and heavily concentrated on the eastern seaboard. Australia's capital is Canberra, and its largest city is Sydney. The country's other major metropolitan areas are Melbourne, Brisbane, Perth, and Adelaide. Indigenous Australians inhabited the continent for about 65,000 years prior to European discovery with the arrival of Dutch explorers in the early 17th century, who named it New Holland. In 1770, Australia's eastern

### Prune non‑relevant candidates (in‑place)

In [37]:
wn_val_subset['candidates_after_pointwise'] = wn_val_subset['candidates_after_pointwise'].apply(
    lambda cand_list: [c for c in cand_list if c.get("relevant") is True] if isinstance(cand_list, list) else cand_list
)

In [38]:
wn_val_subset['post_pt_len_candidates'] = wn_val_subset['candidates_after_pointwise'].apply(
    lambda x: len(x) if isinstance(x, list) else 0
)

In [53]:
wn_val_subset.to_csv('wn_val_subset_5_2_pointwise_results_v2.csv', index=False)

### Evaluation

In [40]:
wn_val_subset.iloc[0].candidates

[{'mentions': 'australian',
  'wiki_id': 4689264,
  'title': 'Australia',
  'text': "Australia, officially the Commonwealth of Australia, is a sovereign country comprising the mainland of the Australian continent, the island of Tasmania, and numerous smaller islands. It is the largest country in Oceania and the world's sixth-largest country by total area. The neighbouring countries are Papua New Guinea, Indonesia, and East Timor to the north; the Solomon Islands and Vanuatu to the north-east; and New Zealand to the south-east. The population of million is highly urbanised and heavily concentrated on the eastern seaboard. Australia's capital is Canberra, and its largest city is Sydney. The country's other major metropolitan areas are Melbourne, Brisbane, Perth, and Adelaide. Indigenous Australians inhabited the continent for about 65,000 years prior to European discovery with the arrival of Dutch explorers in the early 17th century, who named it New Holland. In 1770, Australia's eastern

In [41]:
wn_val_subset.iloc[0].candidates_after_pointwise

[{'mentions': 'australian',
  'wiki_id': 4689264,
  'title': 'Australia',
  'text': "Australia, officially the Commonwealth of Australia, is a sovereign country comprising the mainland of the Australian continent, the island of Tasmania, and numerous smaller islands. It is the largest country in Oceania and the world's sixth-largest country by total area. The neighbouring countries are Papua New Guinea, Indonesia, and East Timor to the north; the Solomon Islands and Vanuatu to the north-east; and New Zealand to the south-east. The population of million is highly urbanised and heavily concentrated on the eastern seaboard. Australia's capital is Canberra, and its largest city is Sydney. The country's other major metropolitan areas are Melbourne, Brisbane, Perth, and Adelaide. Indigenous Australians inhabited the continent for about 65,000 years prior to European discovery with the arrival of Dutch explorers in the early 17th century, who named it New Holland. In 1770, Australia's eastern

In [55]:
def safe_parse_candidates(x):
    if pd.isna(x):
        return []
    try:
        return json.loads(x)
    except json.JSONDecodeError:
        return ast.literal_eval(x)

def compute_metrics_from_pointwise_csv(csv_path):
    df = pd.read_csv(csv_path, dtype={'gt_wiki_id': 'Int64'})
    df['candidates_after_pointwise'] = df['candidates_after_pointwise'].apply(safe_parse_candidates)

    total_mentions = 0  # Count only mentions that retrieved candidates
    recall_hits = 0
    reduction_ratios = []
    retained_counts = []

    for idx, row in df.iterrows():
        candidates = row['candidates_after_pointwise']
        if not candidates:
            continue

        total_mentions += 1

        original_count = row['pre_pt_len_candidates']
        retained_count = row['post_pt_len_candidates']
        retained = [c for c in candidates if c.get('relevant') is True]
        
        if original_count == 0:
            continue

        survived = any(int(c.get('wiki_id')) == int(row['gt_wiki_id']) and c.get('relevant') for c in candidates)

        recall_hits += int(survived)
        reduction_ratios.append(retained_count / original_count)
        retained_counts.append(retained_count)

    recall_at_filter = recall_hits / total_mentions if total_mentions > 0 else 0.0
    reduction_ratio = np.mean(reduction_ratios) if reduction_ratios else 1.0
    avg_retained_candidates = np.mean(retained_counts) if retained_counts else 0.0

    return {
        "Recall@filter": recall_at_filter,
        "Reduction ratio": float(reduction_ratio),
        "Average retained candidates": float(avg_retained_candidates)
    }

In [51]:
# wn_val_subset[wn_val_subset['gt_wiki_id'].isna()==True]
wn_val_subset = wn_val_subset.dropna(subset=['gt_wiki_id'])

In [52]:
wn_val_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239 entries, 0 to 249
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   article_text                239 non-null    object
 1   date                        239 non-null    object
 2   article_title               239 non-null    object
 3   entity_title                239 non-null    object
 4   entity_salience             239 non-null    int64 
 5   offsets                     239 non-null    object
 6   url                         239 non-null    object
 7   surrounding_context         239 non-null    object
 8   gt_wiki_id                  239 non-null    Int64 
 9   candidates                  227 non-null    object
 10  pre_pt_len_candidates       239 non-null    int64 
 11  candidates_after_pointwise  239 non-null    object
 12  post_pt_len_candidates      239 non-null    int64 
dtypes: Int64(1), int64(3), object(9)
memory usage: 26.4+ KB

In [56]:
metrics = compute_metrics_from_pointwise_csv('wn_val_subset_5_2_pointwise_results_v2.csv')
metrics

{'Recall@filter': 0.7911111111111111,
 'Reduction ratio': 0.760846195731428,
 'Average retained candidates': 34.653333333333336}

In [72]:
with open('wn_val_subset_4_27_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

## Contextual EL

In [73]:
sampling_params = get_sampling_params(max_tokens=200, temperature=0.6, top_p=0.9, stops=["</s>", "\n}"])

In [139]:
wn_val_subset = pd.read_csv('wn_val_subset_4_27_pointwise_results.csv', dtype={'gt_wiki_id': 'Int64'})
wn_val_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              100 non-null    int64  
 1   article_text            100 non-null    object 
 2   date                    100 non-null    object 
 3   article_title           100 non-null    object 
 4   entity_title            100 non-null    object 
 5   entity_salience         100 non-null    int64  
 6   offsets                 100 non-null    object 
 7   url                     100 non-null    object 
 8   surrounding_context     100 non-null    object 
 9   gt_wiki_id              99 non-null     float64
 10  candidates              95 non-null     object 
 11  pre_pt_len_candidates   100 non-null    int64  
 12  post_pt_len_candidates  100 non-null    int64  
dtypes: float64(1), int64(4), object(8)
memory usage: 10.3+ KB


In [140]:
import ast

wn_val_subset['candidates'] = wn_val_subset['candidates'].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) else []
)

In [141]:
COT_POOL = [
    {
        "mention": "Washington, D.C.",
        "surrounding_context": "The United States capital of ###Washington, D.C.### legalized same-sex marriage on Wednesday.",
        "candidates": [
            {"id": 1, "name": "Washington, D.C.", "summary": "The capital city of the United States."},
            {"id": 2, "name": "Washington (state)", "summary": "A state in the Pacific Northwest region of the United States."}
        ],
        "answer": "1.Washington, D.C."
    },
    {
        "mention": "Eric Massa",
        "surrounding_context": "New York Representative ###Eric Massa### announced Wednesday that he would be stepping down as Congressman from New York's 29th congressional district.",
        "candidates": [
            {"id": 1, "name": "Eric Massa", "summary": "A former U.S. Congressman from New York."},
            {"id": 2, "name": "Eric Massan", "summary": "A fictional character in a novel."}
        ],
        "answer": "1.Eric Massa"
    },
    {
        "mention": "Air Canada",
        "surrounding_context": "Canadian airline ###Air Canada### has said that it will be laying off 1,010 machinists seconded to work at the Aveos Fleet Performance aircraft maintenance company.",
        "candidates": [
            {"id": 1, "name": "Air Canada", "summary": "The flag carrier and largest airline of Canada."},
            {"id": 2, "name": "Canada Air", "summary": "A fictional airline in a movie."}
        ],
        "answer": "1.Air Canada"
    },
    {
        "mention": "Bosnian president",
        "surrounding_context": "Former ###Bosnian president### Dr. Ejup Ganić was arrested by the Metropolitan Police at Heathrow Airport, London on Monday.",
        "candidates": [
            {"id": 1, "name": "Ejup Ganić", "summary": "A former president of the Federation of Bosnia and Herzegovina."},
            {"id": 2, "name": "Alija Izetbegović", "summary": "The first president of Bosnia and Herzegovina."}
        ],
        "answer": "1.Ejup Ganić"
    },
    {
        "mention": "Greek bailout",
        "surrounding_context": "The spike lower this morning indicates market nervousness about the prospects of a ###Greek bailout### - the message coming out of Europe is still confused.",
        "candidates": [
            {"id": 1, "name": "Greek bailout", "summary": "A financial assistance program for Greece during its debt crisis."},
            {"id": 2, "name": "Greek mythology", "summary": "A body of myths originally told by the ancient Greeks."}
        ],
        "answer": "1.Greek bailout"
    }
]

In [134]:
import random
import textwrap

def contextual_el_chat(entity_row):
    ex = random.choice(COT_POOL)

    ex_candidates = "; ".join([f"{c['id']}.{c['name']} – {c['summary']}" for c in ex['candidates']])

    return [
        {
            "role": "user",
            "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text.  5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text.  Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Now, I will first provide you with an example to illustrate the task: Mention: {ex['mention']} Context: {ex['surrounding_context']} Candidates: {ex_candidates} Answer: {ex['answer']}. Now, I will give you a new mention, its context, and a list of candidate entities. The mention is highlighted with '###'. Mention: {entity_row['entity_title']} Context: {entity_row['surrounding_context']} Candidates: """ + "; ".join([f"{idx + 1}.{cand['title']} – {cand['text'][:100]}"  for idx, cand in enumerate(entity_row['candidates'])]) + """ Think step by step. At the end output exactly one line with the ID and name of the chosen entity, e.g., '3.Barack Obama'. If none of the candidates are correct, output '-1.None'. Only output a single JSON object in the following format, without line breaks, indentation, or extra text: {"final_decision": "<id>.<entity name>", "reasoning": "your explanation here"} """)
        }
    ]

In [135]:
c_prompt = contextual_el_chat(wn_val_subset.iloc[0])

In [136]:
op = llm.chat(messages=c_prompt, sampling_params=sampling_params)
op[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.97it/s, est. speed input: 1047.12 toks/s, output: 118.54 toks/s]


'{"final_decision": "1.Australia", "reasoning": "The context of the text is about a sports event involving the Australian Gliders, indicating that the entity is a country. This is supported by the fact that the Gliders are a national team representing Australia in wheelchair basketball."}'

In [137]:
parse_contextual_el_output(op[0].outputs[0].text)

1

In [138]:
def parse_contextual_el_output(text):
    try:
        match = re.search(r'\{.*?\}', text, re.DOTALL)
        if not match:
            return -1
        
        data = json.loads(match.group())
        decision = data.get("final_decision", "").strip()
        
        # Expecting something like "3.Tesla, Inc."
        m = re.match(r'(-?\d+)\s*\.\s*(.+)', decision)
        if not m:
            return -1
        
        label = int(m.group(1))
        return label

    except Exception as e:
        print(f"[parse_contextual_el_output] Parsing error: {e}")
        return -1

In [142]:
prompt_records = []

for idx, row in wn_val_subset.iterrows():
    if not row['candidates']:
        continue
    prompt = contextual_el_chat(row)
    label_map = {i+1: cand for i, cand in enumerate(row['candidates'])}
    prompt_records.append((idx, label_map, prompt))

In [143]:
BATCH_SIZE = 7500

all_outputs = []

for chunk_start in range(0, len(prompt_records), BATCH_SIZE):
    batch = prompt_records[chunk_start:chunk_start + BATCH_SIZE]
    prompts = [rec[2] for rec in batch]

    responses = llm.chat(messages=prompts, sampling_params=sampling_params)

    for (record, response) in zip(batch, responses):
        idx, label_map, _ = record
        text = response.outputs[0].text.strip()
        selected_label = parse_contextual_el_output(text)

        if selected_label > 0 and selected_label in label_map:
            selected_candidate = label_map[selected_label]
        else:
            selected_candidate = 0

        all_outputs.append((idx, selected_candidate))

Processed prompts: 100%|██████████| 87/87 [00:04<00:00, 18.50it/s, est. speed input: 10753.50 toks/s, output: 2454.74 toks/s]


In [144]:
# top_linked_entities = [None] * len(wn_val_subset)

# for idx, selected_candidate in all_outputs:
#     if selected_candidate:
#         top_linked_entities[idx] = selected_candidate['wiki_id']
#     else:
#         top_linked_entities[idx] = 0

# wn_val_subset['top_linked_entity'] = top_linked_entities

top_linked_entities = [0] * len(wn_val_subset)

for idx, selected_candidate in all_outputs:
    if selected_candidate:
        top_linked_entities[idx] = int(selected_candidate['wiki_id'])

wn_val_subset['top_linked_entity'] = pd.Series(top_linked_entities, dtype="Int64")

In [145]:
wn_val_subset.to_csv('wn_val_subset_4_27_contextual_results.csv', index=False)

In [146]:
def evaluate_contextual_linking(df):
    """
    Evaluate performance of contextual entity linker.
    Assumes 'top_linked_entity' and 'gt_wiki_id' are both integers, with 0 meaning "no prediction made".
    """

    total_mentions = len(df)
    linked_mentions = (df['top_linked_entity'] != 0).sum()
    ground_truth_mentions = df['gt_wiki_id'].notna().sum()

    correct_links = 0

    for _, row in df.iterrows():
        pred = row['top_linked_entity']
        gt = row['gt_wiki_id']

        if pred == 0 or pd.isna(gt):
            continue

        if int(pred) == int(gt):
            correct_links += 1

    accuracy = correct_links / total_mentions if total_mentions else 0
    precision_at_linked = correct_links / linked_mentions if linked_mentions else 0
    recall = correct_links / ground_truth_mentions if ground_truth_mentions else 0
    
    if precision_at_linked + recall > 0:
        f1_score = 2 * (precision_at_linked * recall) / (precision_at_linked + recall)
    else:
        f1_score = 0.0

    print(f"Entities evaluated: {total_mentions}")
    print(f"Ground truth (non-null): {ground_truth_mentions}")
    print(f"Predictions made: {linked_mentions}")
    print(f"Correct links: {correct_links}\n")

    return {
        "Accuracy": accuracy,
        "Precision@Linked": float(precision_at_linked),
        "Recall": float(recall),
        "F1 Score": float(f1_score)
    }


In [147]:
contextual_metrics = evaluate_contextual_linking(wn_val_subset)
contextual_metrics

Entities evaluated: 100
Ground truth (non-null): 99
Predictions made: 59
Correct links: 42



{'Accuracy': 0.42,
 'Precision@Linked': 0.711864406779661,
 'Recall': 0.42424242424242425,
 'F1 Score': 0.5316455696202532}

In [131]:
def error_analysis_contextual_linking(df):
    """
    Analyzes different categories of prediction outcomes:
    - No candidates / No prediction (top_linked_entity == 0)
    - Wrong predictions
    - Correct predictions
    """

    total_mentions = len(df)

    no_prediction = (df['top_linked_entity'] == 0).sum()

    correct_links = 0
    wrong_links = 0

    for _, row in df.iterrows():
        pred = row['top_linked_entity']
        gt = row['gt_wiki_id']

        if pred == 0 or pd.isna(gt):
            continue

        if int(pred) == int(gt):
            correct_links += 1
        else:
            wrong_links += 1

    print(f"Total mentions                : {total_mentions}")
    print(f"No prediction (top_linked_entity == 0) : {no_prediction}")
    print(f"Correct predictions           : {correct_links}")
    print(f"Wrong predictions              : {wrong_links}")

    no_prediction_percent = no_prediction / total_mentions * 100
    correct_percent = correct_links / total_mentions * 100
    wrong_percent = wrong_links / total_mentions * 100

    print("\nPercentage breakdown:")
    print(f"No prediction                  : {no_prediction_percent:.2f}%")
    print(f"Correct prediction              : {correct_percent:.2f}%")
    print(f"Wrong prediction                : {wrong_percent:.2f}%")

    return {
        "No prediction (%)": no_prediction_percent,
        "Correct prediction (%)": correct_percent,
        "Wrong prediction (%)": wrong_percent
    }


In [132]:
error_analysis_contextual_linking(wn_val_subset)

Total mentions                : 100
No prediction (top_linked_entity == 0) : 38
Correct predictions           : 45
Wrong predictions              : 17

Percentage breakdown:
No prediction                  : 38.00%
Correct prediction              : 45.00%
Wrong prediction                : 17.00%


{'No prediction (%)': np.float64(38.0),
 'Correct prediction (%)': 45.0,
 'Wrong prediction (%)': 17.0}